In [26]:
import pandas as pd
import sys
sys.path.append("/root/workSpace/investProject/ConvertBondWheel")

In [27]:
from policy.double_low_enhance.alarm import *

In [28]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('float_format', lambda x: '%.2f' % x)
logging.basicConfig(level=logging.INFO)

In [29]:
#对应配置 /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/config.py
double_low_conf = {
    'db_topk': 0.1,
    'min_curr_iss_amt': 9,  # <剩余规模
    'volatility': 0.1,  # >波动率
    'premium_rt': 0.2,  # <溢价率
    'min_price': 115,  # <可转债价格
    'min_dblow': 125,  # <双低价格
    'redeem': ['Y'],  #  Y=强赎 不在强赎
    'min_year_left': 1,  # >剩余到期年限 (不要一年内到期的)
    'ytm_rt': -0.01,  # > 到期收益率
    'pluse_out_step':10, # 脉冲轮出时， 轮出债比轮入新债双低值大多少
    'pulse_out_price_median':110, # 脉冲轮出时， 市场价格中位数
    'pulse_out_price':[120, 125], # 脉冲轮出时< 110 和>110 对应的轮出价格
    'pulse_out_dblow':[125, 130],  #同上， 双低值
    'user_zh_columns':[u'转债名称', u'双低', u'现价', u'溢价率', u'剩余规模', u'波动率', u'涨跌幅', u'成交额(万元)', u'换手率'],
} 

#user_zh_columns
# 展示字段， 可以继续添加，
# 如果报错， 说明没有爬取到该字段， 可以通过以下步骤来新增：
# 1. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/fields_config.py
# jisilu_dict 新增对应key-value, jisulu_used_columns 新增对应英文名
# 2. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/items.py
# 添加  该字段 = scrapy.Field()

### 周期轮动

In [30]:
# 读当前可转债数据
bond = read_data()
balance_names = read_balancing()

#轮动策略
in_names, out_names = periodWheel(bond, balance_names, double_low_conf)
print("轮入: %d支， 轮出: %d支" %(len(in_names), len(out_names)))

# 结果换成中文header & 按列排序
result = to_display(bond, balance_names, in_names, out_names, user_zh_columns=double_low_conf['user_zh_columns'])
result

INFO:root:dblow Low: 128.280000, dblow high: 130.860000, last db:125.450000


轮入: 7支， 轮出: 0支


/root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/alarm.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_bond['bond_nm'] = d_bond['bond_nm'].apply(lambda s: mark_bond(s, balance_names, in_names, out_names))


,转债名称,双低,现价,溢价率,剩余规模,波动率,涨跌幅,成交额(万元),换手率,代码,正股名称,正股价,正股涨跌,PB,转股价,转股价值,评价,回售触发价,强赎触发价,转债占比,转股起始日,到期时间,剩余年限,到期税前收益,到期税后收益,强赎公告
0,*文科转债,109.96,98.10,11.86%,9.50,26.68%,-0.06%,1009.85,108.00%,128127,文科园林,4.28,0.00%,0.89,4.88,87.70,AA-,3.42,6.34,49.70%,2021-03-01,26-08-19,5.10,4.36%,3.60%,X
2,*孚日转债,111.91,103.42,8.49%,6.50,28.22%,-0.45%,1232.21,183.00%,128087,孚日股份,4.29,-0.69%,1.07,4.50,95.33,AA-,3.15,5.85,16.70%,2020-06-23,25-12-17,4.43,2.04%,1.49%,X
3,*鸿达转债,112.16,107.81,4.35%,7.78,48.10%,-0.82%,2005.94,238.00%,128085,鸿达兴业,4.04,-1.46%,1.41,3.91,103.32,B,2.74,5.08,6.40%,2020-06-22,25-12-16,4.43,3.36%,2.40%,X
4,*利群转债,113.50,100.97,12.53%,18.00,23.96%,-0.13%,861.04,47.00%,113033,利群股份,6.29,0.16%,1.11,7.01,89.73,AA,4.91,9.11,34.30%,2020-10-09,26-03-31,4.71,2.83%,2.24%,X
5,*本钢转债,114.34,101.80,12.54%,67.51,41.28%,0.48%,5692.97,83.00%,127018,本钢板材,4.55,0.22%,0.81,5.03,90.46,AAA,3.52,6.54,42.60%,2021-01-04,26-06-28,4.96,4.87%,3.88%,X
6,*靖远转债,114.65,102.10,12.55%,28.00,44.20%,0.53%,3966.26,139.00%,127027,靖远煤电,2.93,0.00%,0.79,3.23,90.71,AA+,2.26,4.20,41.80%,2021-06-16,26-12-09,5.41,2.32%,1.80%,X
9,*东湖转债,119.15,106.31,12.84%,15.50,30.00%,0.09%,1376.85,84.00%,110080,东湖高新,5.70,-1.04%,0.84,6.05,94.21,AA,4.24,7.87,35.00%,2021-10-18,27-04-12,5.75,1.42%,0.94%,X
10,*绿茵转债,119.28,109.06,10.22%,7.12,36.30%,-0.04%,6416.42,823.00%,127034,绿茵生态,12.25,-2.47%,1.78,12.38,98.95,AA-,8.67,16.09,36.00%,2021-11-11,27-04-29,5.79,1.55%,0.96%,X
11,*湖盐转债,120.27,110.89,9.38%,7.20,30.22%,0.50%,967.57,122.00%,110071,雪天盐业,6.61,1.07%,2.23,6.52,101.38,AA,4.56,8.48,11.90%,2021-01-18,26-07-09,4.99,0.73%,0.19%,X
16,*英特转债,120.88,109.32,11.56%,6.00,64.81%,-0.53%,1059.44,160.00%,127028,英特集团,13.13,0.08%,1.80,13.40,97.99,AA+,9.38,17.42,22.00%,2021-07-12,27-01-04,5.48,0.63%,0.20%,X


In [6]:
result.describe()

,双低,现价,剩余规模,成交额(万元),代码,正股价,PB,转股价,转股价值,强赎触发价,剩余年限
count,26.00,26.00,26.00,26.00,26.00,26.00,26.00,26.00,26.00,26.00,26.00
mean,117.04,105.28,8.14,2880.45,120320.31,8.08,1.77,8.59,94.47,11.17,5.08
std,4.69,3.94,5.51,4255.14,6820.93,3.32,0.71,3.61,6.95,4.70,0.69
min,108.59,99.37,2.45,162.44,110071.00,3.07,0.83,3.23,81.61,4.20,2.73
25%,113.19,103.00,5.29,965.25,113568.50,5.82,1.25,6.23,89.65,8.10,4.76
50%,118.15,104.79,7.16,1449.38,123084.50,8.05,1.70,8.68,93.08,11.29,5.20
75%,119.83,106.87,8.95,2543.44,127031.75,10.08,2.09,10.58,98.89,13.75,5.50
max,124.24,116.53,28.00,20839.11,128127.00,14.63,3.79,16.98,110.44,22.07,5.88


### 脉冲轮动

In [11]:
# 立刻执行爬虫（集思录 & 雪球爬虫）
! sh /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/run_spider.sh

---------scrapy---------
2021-06-29 09:39:34 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: ConvertBondWheel)
2021-06-29 09:39:34 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 07:30:14) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.1.1, Platform Linux-3.10.0-862.el7.x86_64-x86_64-with-glibc2.10
2021-06-29 09:39:34 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'ConvertBondWheel',
 'CONCURRENT_REQUESTS': 64,
 'CONCURRENT_REQUESTS_PER_DOMAIN': 64,
 'CONCURRENT_REQUESTS_PER_IP': 64,
 'LOG_LEVEL': 'INFO',
 'NEWSPIDER_MODULE': 'ConvertBondWheel.spiders',
 'SPIDER_MODULES': ['ConvertBondWheel.spiders']}
2021-06-29 09:39:34 [scrapy.extensions.telnet] INFO: Telnet Password: 49ede7a7ba17f88c
2021-06-29 09:39:34 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetCons

In [12]:
# 轮动策略
bond = read_data()
balance_names = read_balancing()

in_names, out_names = pulseWheel(bond, balance_names, double_low_conf)
print("轮入: %d支， 轮出: %d支" %(len(in_names), len(out_names)))
result = to_display(bond, balance_names, in_names, out_names, user_zh_columns=double_low_conf['user_zh_columns'])
result

轮入: 0支， 轮出: 0支


/root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/alarm.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_bond['bond_nm'] = d_bond['bond_nm'].apply(lambda s: mark_bond(s, balance_names, in_names, out_names))


,转债名称,双低,现价,溢价率,剩余规模,波动率,涨跌幅,成交额(万元),换手率,代码,正股名称,正股价,正股涨跌,PB,转股价,转股价值,评价,回售触发价,强赎触发价,转债占比,转股起始日,到期时间,剩余年限,到期税前收益,到期税后收益,强赎公告
0,*文科转债,109.53,98.99,10.54%,9.50,28.04%,0.16%,80.51,9.00%,128127,文科园林,4.37,-0.23%,0.91,4.88,89.55,AA-,3.42,6.34,48.90%,2021-03-01,26-08-19,5.14,4.13%,3.38%,X
2,*靖远转债,111.95,101.53,10.42%,28.00,45.26%,0.18%,293.38,10.00%,127027,靖远煤电,2.97,0.34%,0.80,3.23,91.95,AA+,2.26,4.20,41.30%,2021-06-16,26-12-09,5.45,2.41%,1.89%,X
3,*孚日转债,111.96,102.95,9.01%,6.50,29.51%,-0.20%,42.75,6.00%,128087,孚日股份,4.25,0.00%,1.06,4.50,94.44,AA-,3.15,5.85,16.90%,2020-06-23,25-12-17,4.47,2.13%,1.59%,X
4,*鸿达转债,112.34,107.51,4.83%,7.78,48.45%,-0.19%,236.16,28.00%,128085,鸿达兴业,4.01,-1.23%,1.40,3.91,102.56,B,2.74,5.08,6.50%,2020-06-22,25-12-16,4.47,3.40%,2.44%,X
5,*东湖转债,113.93,106.61,7.32%,15.50,30.63%,-0.06%,129.72,8.00%,110080,东湖高新,6.01,0.00%,0.88,6.05,99.34,AA,4.24,7.87,33.20%,2021-10-18,27-04-12,5.79,1.36%,0.88%,X
6,*利群转债,114.25,101.07,13.18%,18.00,24.26%,0.10%,38.31,2.00%,113033,利群股份,6.26,-0.32%,1.10,7.01,89.30,AA,4.91,9.11,34.50%,2020-10-09,26-03-31,4.76,2.79%,2.20%,X
8,*湖盐转债,115.99,108.98,7.01%,7.20,31.35%,0.24%,380.12,48.00%,110071,雪天盐业,6.64,0.61%,2.20,6.52,101.84,AA,4.56,8.48,11.80%,2021-01-18,26-07-09,5.03,1.13%,0.58%,X
11,*灵康转债,116.94,106.61,10.33%,5.25,37.59%,-0.26%,66.67,12.00%,113610,灵康药业,8.32,-0.36%,3.92,8.61,96.63,AA-,6.03,11.19,8.80%,2021-06-07,26-11-30,5.42,2.43%,1.74%,X
14,*美力转债,118.32,114.39,3.93%,3.00,53.07%,0.44%,69.96,20.00%,123097,美力科技,10.28,2.19%,2.40,9.34,110.06,A+,6.54,12.14,24.00%,2021-08-02,27-01-26,5.58,1.13%,0.46%,X
15,*英特转债,118.43,108.56,9.87%,6.00,65.41%,-0.28%,15.64,2.00%,127028,英特集团,13.24,-1.05%,1.82,13.40,98.81,AA+,9.38,17.42,21.90%,2021-07-12,27-01-04,5.52,0.76%,0.32%,X
